In [21]:
%reset -f
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt

#known parameters
M = 0.8
altitude = 35000 #in feet
pressure = 497.959 #in pounds-force/sq ft
air_density = 0.000736627 #in slugs/ft^3
speed_sound = 972.885 #in ft/sec
dynamic_viscosity = 3.01681e-7 #in pound-force seconds / ft^2
velocity = M * speed_sound
q = (1 / 2) * air_density * velocity**2

#WINGS
b_wing = 200 #in feet
AR_wing = 9
sweep_wing = 30 #in degrees
thickness_to_chord_ratio_wing = 0.12
S_wing = b_wing**2 / AR_wing
mean_geometric_chord_wing = S_wing / b_wing
S_wet_wing = 2 * (1 + 0.2 * thickness_to_chord_ratio_wing) * S_wing

#HORIZONTAL TAIL
b_horizontal_tail = 70 #in feet
AR_horizontal_tail = 4
sweep_horizontal_tail = 25 #in degrees
thickness_to_chord_ratio_horizontal_tail = 0.1
S_horizontal_tail = b_horizontal_tail**2 / AR_horizontal_tail
mean_geometric_chord_horizontal_tail = S_horizontal_tail / b_horizontal_tail
S_wet_horizontal_tail = 2 * (1 + 0.2 * thickness_to_chord_ratio_horizontal_tail) * S_wing

#VERTICAL TAIL
b_vertical_tail = 30 #in feet
AR_vertical_tail = 4
sweep_vertical_tail = 35 #in degrees
thickness_to_chord_ratio_vertical_tail = 0.1
S_vertical_tail = b_vertical_tail**2 / AR_vertical_tail
mean_geometric_chord_vertical_tail = S_vertical_tail / b_vertical_tail
S_wet_vertical_tail = 2 * (1 + 0.2 * thickness_to_chord_ratio_vertical_tail) * S_vertical_tail

#FUSELAGE
length_fuselage = 160 #in feet
diameter_fuselage = 20 #in feet
nose_cone_length = 20 #in feet
tail_cone_length = 30 #in feet
S_wet_cylinder = np.pi * diameter_fuselage * length_fuselage
S_wet_nose_cone = 0.75 * np.pi * diameter_fuselage * nose_cone_length
S_wet_tail_cone = 0.75 * np.pi * diameter_fuselage * tail_cone_length
S_wet_fuselage_total = S_wet_cylinder + S_wet_nose_cone + S_wet_tail_cone
fr = (length_fuselage + nose_cone_length + tail_cone_length) / diameter_fuselage

def calculate_wing_drag(air_density,velocity,chord_length,dynamic_viscosity,M,q,sweep_angle,thickness_to_chord_ratio,S_wet):
    Re = air_density * velocity * chord_length / dynamic_viscosity
    cf_inc = 0.074 / Re**0.2
    cf = cf_inc * (1 + 0.144 * M**2)**-0.65
    Z = (2 - M**2) * np.cos(np.deg2rad(sweep_angle)) / np.sqrt(1 - M**2 * np.cos(np.deg2rad(sweep_angle))**2)
    k = 1 + Z * thickness_to_chord_ratio + 100 * (thickness_to_chord_ratio)**4
    drag_force = cf * S_wet * k * q
    return drag_force

#calculate wing drag forces using calculate_wing_drag function
drag_force_wing = calculate_wing_drag(air_density,velocity,mean_geometric_chord_wing,dynamic_viscosity,M,q,sweep_wing,thickness_to_chord_ratio_wing,S_wet_wing)
drag_force_horizontal_tail = calculate_wing_drag(air_density,velocity,mean_geometric_chord_horizontal_tail,dynamic_viscosity,M,q,sweep_horizontal_tail,thickness_to_chord_ratio_horizontal_tail,S_wet_horizontal_tail)
drag_force_vertical_tail = calculate_wing_drag(air_density,velocity,mean_geometric_chord_vertical_tail,dynamic_viscosity,M,q,sweep_vertical_tail,thickness_to_chord_ratio_vertical_tail,S_wet_vertical_tail)

#calculate fuselage drag force (without a function because its only done once)
if (fr > 5) and (fr < 15):
    k_fuselage = 1.675 - 0.09 * fr + 0.003 * fr**2
elif (fr > 15):
    k_fuselage = 1
else:
    k_fuselage = np.nan
    print('ERROR: POOR FINENESS RATIO FOR FUSELAGE')
Re_fuselage = air_density * velocity * (length_fuselage + nose_cone_length + tail_cone_length) / dynamic_viscosity
cf_inc_fuselage = 0.074 / Re_fuselage**0.2
cf_fuselage = cf_inc_fuselage * (1 + 0.144 * M**2)**-0.65
drag_force_fuselage = cf_fuselage * S_wet_fuselage_total * k_fuselage * q

#calculate total drag force
drag_force_total = drag_force_wing + drag_force_horizontal_tail + drag_force_vertical_tail + drag_force_fuselage

#percent drag by parts (for fun)
percent_drag_wings = drag_force_wing / drag_force_total
percent_drag_horizontal_tails = drag_force_horizontal_tail / drag_force_total
percent_drag_vertical_tail = drag_force_vertical_tail / drag_force_total
percent_drag_fuselage = drag_force_fuselage / drag_force_total

#lift to drag ratio
lift_maybe = 90710 #in lbforce
lift_to_drag_ratio = lift_maybe / drag_force_total


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
